In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('dataset_entradas_100k.csv')
df.head(50)

,Evento,Artista,Lugar,Capacidad,Precio Promedio,Promoción,Ventas Pasadas,Fecha
0,Jazz Evening,Artista D,Plaza Central,13825,216,Alta,3252,2024-07-21
1,Classical Evening,Artista C,Arena Grande,22393,268,Media,19630,2024-12-25
2,Pop Festival,Banda A,Estadio 123,6421,367,Media,6421,2024-09-20
3,Jazz Evening,Artista E,Arena Grande,18166,460,Alta,18166,2024-02-12
4,Indie Vibes,Artista D,Arena Grande,22333,497,Baja,22333,2024-03-15
5,Pop Festival,Banda B,Auditorio ABC,35638,172,Media,35638,2024-12-06
6,Indie Vibes,Banda A,Auditorio ABC,42787,53,Alta,42787,2024-03-16
7,Indie Vibes,Banda A,Plaza Central,43532,75,Baja,14196,2024-10-02
8,Jazz Evening,Banda B,Estadio 123,24665,433,Alta,3967,2024-07-30
9,Jazz Evening,Banda B,Plaza Central,9278,385,Media,9278,2024-08-27


In [3]:
# Convertir la columna 'Fecha' a formato datetime y extraer características
df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Año'] = df['Fecha'].dt.year
df['Mes'] = df['Fecha'].dt.month
df['Día_Semana'] = df['Fecha'].dt.weekday  # 0=Lunes, 6=Domingo
df['Días_Hasta_Evento'] = (df['Fecha'] - pd.Timestamp.today()).dt.days
df = df.drop(columns=['Fecha'])

In [4]:

# Separar las características (X) de la variable objetivo (y)
X = df.drop(columns=['Ventas Pasadas'])  # Elimina la columna objetivo (Ventas Realizadas)
y = df['Ventas Pasadas']

# Definir las columnas categóricas para One-Hot Encoding
categorical_columns = ['Evento', 'Artista', 'Lugar', 'Promoción']

# Usar ColumnTransformer para aplicar OneHotEncoder a las columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ], remainder='passthrough'  # Deja el resto de las columnas (numéricas) tal como están
)

In [8]:
# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# Aplicar el preprocesador (OneHotEncoding) al conjunto de entrenamiento
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)  # Aplicar el mismo transformador al conjunto de prueba

# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

LinearRegression()

In [9]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: {rmse}")

# Evaluar la exactitud del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")

# Evaluo la precision del modelo
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Precisión del modelo: {precision:.2f}")

# Evaluo el recall del modelo
recall = recall_score(y_test, y_pred, average='weighted')
print(f"Recall del modelo: {recall:.2f}")

# Evaluo el f1 score del modelo
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score del modelo: {f1:.2f}")

RMSE: 9358.03073698796


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [11]:
# Ejemplo de predicción para un nuevo evento
nuevo_evento = {
    'Evento': 'Monster Rock',
    'Artista': 'Metallica',
    'Lugar': 'Teatro Vorterix',
    'Capacidad': 10000,
    'Precio Promedio': 5000,
    'Promoción': 'Alta',
    'Ventas Pasadas': 8000,
    'Año': 2024,
    'Mes': 10,
    'Día_Semana': 3,
    'Días_Hasta_Evento': 15  # Faltan 15 días para el evento
}

# Convertir el nuevo evento a un DataFrame
nuevo_evento_df = pd.DataFrame([nuevo_evento])

# Aplicar el mismo preprocesamiento (OneHotEncoding) al nuevo evento
nuevo_evento_transformed = preprocessor.transform(nuevo_evento_df)

# Realizar la predicción
ventas_predichas = model.predict(nuevo_evento_transformed)
print(f"Predicción de ventas para el nuevo evento: {ventas_predichas[0]:.0f} entradas")

Predicción de ventas para el nuevo evento: 8953 entradas


In [10]:
import joblib
joblib.dump(model, 'modelo_regresion_entradas.pkl')


['modelo_regresion_entradas.pkl']

In [11]:
joblib.dump(preprocessor, 'preprocesador.pkl')

['preprocesador.pkl']